In [ ]:
!pip install --upgrade --quiet langchain langchain-community langchain-huggingface chromadb gradio sentence-transformers pandas tabulate

In [ ]:
!pip uninstall pandas

In [ ]:
!pip install pandas


In [ ]:
!pip install tabulate


In [ ]:
import os
import ast
import re
import pandas as pd
from langchain_community.utilities import SQLDatabase
from langchain_community.vectorstores import Chroma
from langchain_community.agent_toolkits import create_sql_agent
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import ChatPromptTemplate, FewShotPromptTemplate, MessagesPlaceholder, PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
#from langchain_community.llms import HuggingFaceHub
from langchain_huggingface import HuggingFaceEndpoint
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.tools.base import Tool
import gradio as gr
from google.colab import drive
import getpass
from tabulate import tabulate

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Đường dẫn đến database
sql_path = '/content/drive/MyDrive/chat/products.db'

In [ ]:
# Khởi tạo SQL Database
db = SQLDatabase.from_uri(f"sqlite:///{sql_path}")
print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
# Lấy schema chi tiết hơn để cung cấp cho LLM
try:
    db_schema = db.get_table_info(db.get_usable_table_names())
    print(f"Schema:\n{db_schema}")
except Exception as e:
    print(f"Error getting detailed schema: {e}")
    db_schema = "Không thể lấy schema chi tiết."

In [ ]:
import os
import getpass
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass.getpass('Hugging Face Hub API Token:')

In [ ]:
# Khởi tạo Hugging Face LLM
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    huggingfacehub_api_token=os.environ.get('HUGGINGFACEHUB_API_TOKEN'),
    temperature=0.1,
    max_new_tokens=600,
    top_p=0.9,
    repetition_penalty=1.1,

)

Liệt,  kê ,  các ,  sản ,  phẩm ?
Select

In [ ]:
# Khởi tạo embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"  #Dùng để tìm kiếm văn bản tương tự (semantic search),
)

In [ ]:
#Các ví dụ mẫu
examples = [
    {"input": "Có bao nhiêu danh mục sản phẩm?", "query": "SELECT COUNT(DISTINCT id) FROM categories;"},
    {"input": "Liệt kê tên các danh mục.", "query": "SELECT name FROM categories;"},
    {"input": "Những sản phẩm nào thuộc danh mục 'Điện Tử'?", "query": "SELECT p.name, p.price FROM products p JOIN categories c ON p.category_id = c.id WHERE c.name = 'Điện Tử' LIMIT 10;"},
    {"input": "Cho tôi xem 5 sản phẩm đắt nhất.", "query": "SELECT name, price FROM products ORDER BY price DESC LIMIT 5;"},
    {"input": "Tìm các sản phẩm có giá dưới 200 nghìn.", "query": "SELECT name, price FROM products WHERE price < 200000 ORDER BY price ASC LIMIT 10;"},
    {"input": "Sản phẩm nào có giá từ 500.000 đến 1 triệu đồng?", "query": "SELECT name, price FROM products WHERE price >= 500000 AND price <= 1000000 ORDER BY price LIMIT 10;"},
    {"input": "Có sản phẩm nào tên là 'Laptop ABC' không?", "query": "SELECT name, price, stock_quantity FROM products WHERE name = 'Laptop ABC';"},
    {"input": "Sản phẩm rẻ nhất trong danh mục 'Sách' là gì?", "query": "SELECT p.name, p.price FROM products p JOIN categories c ON p.category_id = c.id WHERE c.name = 'Sách' ORDER BY p.price ASC LIMIT 1;"},
    {"input": "Số lượng tồn kho của 'Áo Thun XYZ' là bao nhiêu?", "query": "SELECT stock_quantity FROM products WHERE name = 'Áo Thun XYZ';"},
    {"input": "Danh sách các sản phẩm sắp hết hàng (tồn kho < 10).", "query": "SELECT name, stock_quantity FROM products WHERE stock_quantity < 10 ORDER BY stock_quantity ASC;"},
    {"input": "Tổng giá trị của tất cả sản phẩm trong kho là bao nhiêu?", "query": "SELECT SUM(price * stock_quantity) FROM products;"},
    {"input": "Danh mục nào có nhiều sản phẩm nhất?", "query": "SELECT c.name, COUNT(p.id) as product_count FROM categories c JOIN products p ON c.id = p.category_id GROUP BY c.id, c.name ORDER BY product_count DESC LIMIT 1;"},
    {"input": "Hiển thị các sản phẩm có từ 'thông minh' trong tên.", "query": "SELECT name, price FROM products WHERE name LIKE '%thông minh%' LIMIT 10;"},
    {"input": "Giá trung bình của các sản phẩm trong danh mục 'Gia Dụng'?", "query": "SELECT AVG(p.price) FROM products p JOIN categories c ON p.category_id = c.id WHERE c.name = 'Gia Dụng';"},
    {"input": "Liệt kê các sản phẩm được thêm vào sau ngày '2024-01-01' (nếu có cột ngày tạo).", "query": "SELECT name, created_at FROM products WHERE created_at > '2024-01-01';"}, # Giả sử có cột created_at
    {"input": "Những sản phẩm nào không thuộc danh mục 'Thời Trang'?", "query": "SELECT p.name FROM products p LEFT JOIN categories c ON p.category_id = c.id WHERE c.name IS NULL OR c.name != 'Thời Trang' LIMIT 10;"},
    {"input": "Cho xem 3 sản phẩm bất kỳ.", "query": "SELECT name, price FROM products ORDER BY RANDOM() LIMIT 3;"},
    {"input": "Mô tả sản phẩm 'Điện thoại Z Ultra' là gì?", "query": "SELECT description FROM products WHERE name = 'Điện thoại Z Ultra';"}, # Giả sử có cột description
    {"input": "Tìm sản phẩm có giá chính xác là 155000 đồng.", "query": "SELECT name, price FROM products WHERE price = 155000;"},
    {"input": "Liệt kê tất cả sản phẩm và danh mục tương ứng của chúng.", "query": "SELECT p.name AS product_name, c.name AS category_name FROM products p JOIN categories c ON p.category_id = c.id LIMIT 20;"},
    {"input": "Các sản phẩm nào có giá lớn hơn 1.000.000 và tồn kho nhiều hơn 5?", "query": "SELECT name, price, stock_quantity FROM products WHERE price > 1000000 AND stock_quantity > 5;"},
    {"input": "Có bao nhiêu sản phẩm trong mỗi danh mục?", "query": "SELECT c.name, COUNT(p.id) as number_of_products FROM categories c LEFT JOIN products p ON c.id = p.category_id GROUP BY c.id, c.name ORDER BY number_of_products DESC;"},
    {"input": "Sản phẩm nào có chữ 'áo' hoặc 'quần' trong tên và giá dưới 500k?", "query": "SELECT name, price FROM products WHERE (name LIKE '%áo%' OR name LIKE '%quần%') AND price < 500000;"},
    {"input": "Sản phẩm có giá cao nhất và thấp nhất là gì?", "query": "(SELECT name, price, 'Cao nhất' as type FROM products ORDER BY price DESC LIMIT 1) UNION ALL (SELECT name, price, 'Thấp nhất' as type FROM products ORDER BY price ASC LIMIT 1);"}
]

In [ ]:
# Khởi tạo example selector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=5,
    input_keys=['input']
)


In [ ]:
# Hàm tiền xử lý câu hỏi
def preprocess_query(query_text):
    query_text = query_text.lower().strip()
    # Giữ lại dấu tiếng Việt và một số ký tự đặc biệt cơ bản cho giá tiền, tên
    query_text = re.sub(r'[^\w\sàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ\.,\-\'$€£¥]', '', query_text)
    return query_text


In [ ]:
#  Hàm phân tích loại truy vấn
def analyze_query_type(query_text):
    query_text_lower = query_text.lower()

    # Nhận diện các thành phần chính của câu hỏi
    is_listing = any(k in query_text_lower for k in ["liệt kê", "cho xem", "tìm", "hiển thị", "danh sách"])
    is_counting = "bao nhiêu" in query_text_lower or "đếm số" in query_text_lower
    is_summing = "tổng giá trị" in query_text_lower # Thêm cho SUM
    is_averaging = "trung bình giá" in query_text_lower or "giá trung bình" in query_text_lower # Thêm cho AVG

    has_product_keyword = "sản phẩm" in query_text_lower
    has_category_keyword = "danh mục" in query_text_lower

    has_price_less_than = "giá dưới" in query_text_lower or "giá <" in query_text_lower
    has_price_greater_than = "giá trên" in query_text_lower or "giá >" in query_text_lower
    has_price_range = ("giá từ" in query_text_lower and "đến" in query_text_lower) or ("khoảng giá" in query_text_lower)
    has_price_exact = "giá chính xác" in query_text_lower or "giá bằng" in query_text_lower
    is_price_highest = "cao nhất" in query_text_lower or "đắt nhất" in query_text_lower
    is_price_lowest = "thấp nhất" in query_text_lower or "rẻ nhất" in query_text_lower

    category_filter_match = re.search(r"(?:thuộc|trong)\s+danh mục\s+['\"]?([^'\"]+)['\"]?", query_text_lower)
    has_category_filter = category_filter_match is not None

    product_name_exact_match = re.search(r"(?:tên là|sản phẩm là|tên của)\s*['\"]?([^'\"]+)['\"]?", query_text_lower)
    has_product_name_filter_exact = product_name_exact_match is not None
    product_name_like_match = re.search(r"(?:có từ|chứa từ|tên có)\s*['\"]?([^'\"]+)['\"]?\s*(?:trong tên)?", query_text_lower)
    has_product_name_filter_like = product_name_like_match is not None

    has_stock_query = "tồn kho" in query_text_lower or "còn lại" in query_text_lower
    # "số lượng" có thể hỏi về số lượng sản phẩm hoặc số lượng tồn kho
    is_stock_count_for_product = has_stock_query and (has_product_name_filter_exact or \
                                 (product_name_exact_match := re.search(r"(?:của|cho)\s*['\"]?([^'\"]+)['\"]?", query_text_lower)))


    # ---- Ưu tiên các loại truy vấn phức tạp hoặc cụ thể trước ----
    if is_summing and "sản phẩm trong kho" in query_text_lower: # "Tổng giá trị của tất cả sản phẩm trong kho"
        return "total_stock_value"

    if is_counting and has_category_keyword and not has_product_keyword: # "Có bao nhiêu danh mục sản phẩm?"
        return "count_distinct_categories"

    if is_listing and has_category_keyword and not has_product_keyword and not category_filter_match: # "Liệt kê tên các danh mục."
        return "list_categories"

    # Các câu hỏi kết hợp điều kiện
    if has_price_less_than and has_category_filter:
        return "price_less_than_and_in_category"
    if has_price_greater_than and has_category_filter:
        return "price_greater_than_and_in_category"
    if has_price_range and has_category_filter:
        return "price_range_and_in_category"
    if is_price_lowest and has_category_filter: # "Sản phẩm rẻ nhất trong danh mục X"
        return "price_lowest_in_category"
    if is_price_highest and has_category_filter: # "Sản phẩm đắt nhất trong danh mục X"
        return "price_highest_in_category"

    if is_listing and has_product_keyword and has_price_greater_than and has_stock_query and "nhiều hơn" in query_text_lower:
        # "Các sản phẩm nào có giá lớn hơn X và tồn kho nhiều hơn Y"
        return "price_greater_and_stock_greater"

    if is_stock_count_for_product: # "Số lượng 'Laptop Gaming Pro' còn trong kho"
         # Cập nhật lại product_name_exact_match nếu nó được gán trong điều kiện is_stock_count_for_product
        if not product_name_exact_match and re.search(r"(?:của|cho)\s*['\"]?([^'\"]+)['\"]?", query_text_lower):
            product_name_exact_match = re.search(r"(?:của|cho)\s*['\"]?([^'\"]+)['\"]?", query_text_lower)
        return "stock_query_for_product"

    # Các loại truy vấn đơn giản hơn
    if any(k in query_text_lower for k in ["mô tả", "chi tiết về"]):
        return "product_description"

    if is_counting and has_product_keyword and "mỗi danh mục" in query_text_lower :
        return "count_products_per_category"
    if is_counting and has_product_keyword and has_category_filter: # "bao nhiêu sản phẩm trong danh mục X"
        return "count_products_in_category"
    if is_counting and has_product_keyword: # "bao nhiêu sản phẩm (tổng)"
        return "count_distinct_products"

    if is_averaging and has_category_filter:
        return "average_price_in_category"

    # Chung chung về giá
    if is_price_highest: return "price_highest"
    if is_price_lowest: return "price_lowest"
    if has_price_less_than: return "price_less_than"
    if has_price_greater_than: return "price_greater_than"
    if has_price_range: return "price_range"
    if has_price_exact: return "price_exact"

    # Chung chung về sản phẩm/danh mục
    if is_listing and has_product_name_filter_like: # "Hiển thị các sản phẩm có từ 'thông minh' trong tên."
        return "products_by_name_like"
    if has_product_name_filter_exact: # "Có sản phẩm nào tên là 'Laptop ABC' không?"
        return "product_by_exact_name"
    if has_category_filter : # "Những sản phẩm nào thuộc danh mục 'Điện Tử'"
        return "products_in_category" # Đổi tên từ specific_products_in_category

    # Fallback cuối cùng cho các lệnh liệt kê chung
    if is_listing and has_product_keyword: return "product_list_all"
    if is_listing: return "unknown_listing" # Nếu chỉ có "liệt kê" mà không rõ đối tượng

    return "unknown"

In [ ]:
# Hàm truy vấn dữ liệu và chuyển kết quả về dạng list
def query_as_list(db_conn, query_text):
    print(f"Executing fallback query: {query_text}") # Logging
    try:
        res = db_conn.run(query_text)
        if isinstance(res, str):
            # Cố gắng chuyển đổi chuỗi JSON-like hoặc list-like
            try:
                res_eval = ast.literal_eval(res)
                if isinstance(res_eval, list):
                    return res_eval # Giữ nguyên cấu trúc list của tuples
            except (SyntaxError, ValueError):
                # Nếu không phải list-like, có thể là một giá trị đơn hoặc chuỗi lỗi
                print(f"Query returned a non-list string: {res}")
                return [[res]] # Trả về list of list cho đồng nhất
        elif isinstance(res, list): # Đã là list (có thể là list of tuples)
            return res
        return [[str(res)]] # Xử lý các trường hợp khác
    except Exception as e:
        print(f"Error in query_as_list executing query '{query_text}': {e}")
        return [["Lỗi khi truy vấn cơ sở dữ liệu."]]


In [ ]:
# Lấy danh sách tên sản phẩm, danh mục và các cụm từ tìm kiếm phổ biến
try:
    products_raw = query_as_list(db, "SELECT name FROM products")
    products = [item[0] for item in products_raw if isinstance(item, (list, tuple)) and len(item) > 0]

    categories_raw = query_as_list(db, "SELECT name FROM categories")
    categories = [item[0] for item in categories_raw if isinstance(item, (list, tuple)) and len(item) > 0]

    all_items = list(set(products + categories)) # Loại bỏ trùng lặp

    # Thêm các cụm từ mô tả thuộc tính hoặc giá trị phổ biến
    common_search_terms = [
        "dưới 100 nghìn", "dưới 200k", "dưới 500k", "dưới 1 triệu",
        "từ 100 đến 300 nghìn", "từ 500k đến 1tr", "khoảng 1 triệu đến 2 triệu",
        "trên 1 triệu", "trên 500 nghìn", "giá rẻ", "cao cấp", "mới nhất", "phổ biến",
        # Thêm tên các cột quan trọng nếu người dùng có thể hỏi về chúng
        "tên sản phẩm", "giá", "số lượng tồn kho", "mô tả", "danh mục"
    ]
    all_items.extend(common_search_terms)
    all_items = list(set(all_items)) # Đảm bảo không trùng lặp

    print(f"Đã tải {len(all_items)} tên sản phẩm, danh mục, và cụm từ tìm kiếm cho retriever.")
except Exception as e:
    print(f"Error loading product and category names for retriever: {e}")
    all_items = ["Lỗi tải dữ liệu retriever"]



In [ ]:
# Tạo vectordb cho retriever
if all_items and all_items[0] != "Lỗi tải dữ liệu retriever":
    vector_db_retriever = Chroma.from_texts(all_items, embeddings)
    retriever = vector_db_retriever.as_retriever(search_kwargs={"k": 10})
else:
    # Fallback nếu không tải được all_items
    retriever = Chroma.from_texts(["dữ liệu mẫu"], embeddings).as_retriever()
    print("Cảnh báo: Retriever đang sử dụng dữ liệu mẫu do lỗi tải.")

In [ ]:
#  Mô tả công cụ tìm kiếm
retriever_tool_description = """
Công cụ này CHỈ dùng để tìm kiếm tên CHÍNH XÁC của sản phẩm, danh mục, hoặc các cụm từ liên quan đến thuộc tính sản phẩm trong cơ sở dữ liệu.
Đầu vào là một phần tên hoặc mô tả gần đúng mà người dùng cung cấp.
Đầu ra là một danh sách các tên hoặc cụm từ CHÍNH XÁC có trong cơ sở dữ liệu.
BẮT BUỘC sử dụng tên chính xác từ kết quả của công cụ này trong mệnh đề WHERE của câu truy vấn SQL nếu câu hỏi của người dùng liên quan đến việc lọc theo tên sản phẩm, tên danh mục, hoặc các thuộc tính cụ thể đã được index.
Ví dụ: Nếu người dùng hỏi "tìm lap top XZY", công cụ có thể trả về "Laptop XYZ Model A". Bạn PHẢI dùng "Laptop XYZ Model A" trong truy vấn SQL.
Không dùng công cụ này để trả lời câu hỏi trực tiếp, chỉ dùng để lấy tên/thuật ngữ chính xác cho truy vấn SQL.
"""

In [ ]:
# Tạo retriever tool
retriever_tool = create_retriever_tool(
    retriever,
    name="search_products_categories_attributes",
    description=retriever_tool_description
)
tools = [retriever_tool]

In [ ]:
#  Xử lý giá trị tiền tệ
def extract_price_value(text):
    # Loại bỏ dấu phẩy ngăn cách hàng nghìn để tránh lỗi chuyển đổi
    text_no_comma = text.replace(',', '').replace('.', '')
    numbers = re.findall(r'\d+', text_no_comma)
    if not numbers:
        return None

    # Tìm số lớn nhất, giả sử đó là giá trị chính
    price = 0
    # Ưu tiên các số có vẻ là giá đầy đủ hơn (ví dụ: 300000 thay vì 300)
    potential_prices = sorted([int(n) for n in numbers], reverse=True)

    # Heuristic để xác định giá chính xác hơn
    # Ví dụ: "giá 300 nghìn" -> 300000, "giá 1 triệu 500" -> 1500000
    if "triệu" in text:
        # Tìm số đi liền trước "triệu"
        match_triệu = re.search(r'(\d+)\s*(triệu|tr)', text, re.IGNORECASE)
        if match_triệu:
            price = int(match_triệu.group(1)) * 1000000
            # Kiểm tra xem có "nghìn" sau "triệu" không (vd: 1 triệu 500 nghìn)
            remaining_text = text[match_triệu.end():]
            match_nghìn_after_triệu = re.search(r'(\d+)\s*(nghìn|k|ngàn)', remaining_text, re.IGNORECASE)
            if match_nghìn_after_triệu:
                price += int(match_nghìn_after_triệu.group(1)) * 1000
            return price
    if "nghìn" in text or "ngàn" in text or "k" in text.lower():
        match_nghìn = re.search(r'(\d+)\s*(nghìn|k|ngàn)', text, re.IGNORECASE)
        if match_nghìn:
            price = int(match_nghìn.group(1)) * 1000
            return price

    # Nếu không có "triệu" hay "nghìn" rõ ràng, dùng số lớn nhất
    if potential_prices:
      # Nếu số lớn nhất có vẻ là đơn vị cơ bản (ví dụ > 10000) thì dùng nó
      # Hoặc nếu là số nhỏ (ví dụ 300) thì có thể là nó thiếu "nghìn"
      # Đây là một heuristic,
        if potential_prices[0] > 10000 or len(str(potential_prices[0])) > 3: # vd: 150000 hoặc 300 nếu không có nghìn/triệu
             return potential_prices[0]
        elif len(potential_prices[0]) <=3 and (len(text.split()) < 5) : # vd: "300" trong câu ngắn -> 300000
             return potential_prices[0] * 1000


    return potential_prices[0] if potential_prices else None


In [ ]:
# Hàm định dạng kết quả SQL
def format_sql_result(result_input, query_text_original=""):
    print(f"Formatting result for query: {query_text_original}")
    print(f"Raw result from DB: {result_input}")

    if not result_input:
        return "Không tìm thấy kết quả hoặc truy vấn không trả về dữ liệu."

    data = []
    if isinstance(result_input, str):
        try:
            # Chuỗi có thể là một list các tuple/list dưới dạng string
            # Hoặc một giá trị đơn giản
            evaluated_input = ast.literal_eval(result_input)
            if isinstance(evaluated_input, list):
                data = evaluated_input
            elif isinstance(evaluated_input, (int, float, str, tuple)):
                data = [[evaluated_input]] # Đưa về dạng list of lists
            else:
                data = [[str(result_input)]]
        except (SyntaxError, ValueError) as e:
            print(f"Error evaluating result string '{result_input}': {e}. Treating as single string.")
            # Nếu là một chuỗi đơn giản không phải cấu trúc list
            return str(result_input) # Trả về trực tiếp nếu không parse được
    elif isinstance(result_input, list):
        data = result_input
    elif isinstance(result_input, (int, float, tuple)): # Xử lý trường hợp kết quả là một số hoặc tuple đơn
        data = [[result_input]]
    else:
        return "Định dạng kết quả không được hỗ trợ."

    if not data or not isinstance(data, list) or (isinstance(data[0], list) and not data[0]): # Kiểm tra data rỗng hoặc list chứa list rỗng
        if "COUNT" in query_text_original.upper() and isinstance(data, list) and len(data) > 0 and isinstance(data[0], (list, tuple)) and (data[0][0] == 0 or data[0][0] is None):
             return "Số lượng là 0."
        return "Không có dữ liệu phù hợp với yêu cầu của bạn."


    # Cố gắng lấy tên cột từ câu truy vấn SQL (đây là một heuristic)
    headers = []
    # Tìm các cột được SELECT, tránh các hàm và bí danh phức tạp nếu có thể
    match = re.search(r"SELECT\s+(.*?)\s+FROM", query_text_original, re.IGNORECASE | re.DOTALL)
    if match:
        select_clause = match.group(1)
        # Tách các cột, xử lý bí danh đơn giản (column AS alias)
        possible_headers = re.split(r',\s*', select_clause)
        for ph in possible_headers:
            alias_match = re.search(r'\s+AS\s+(\w+)', ph, re.IGNORECASE)
            if alias_match:
                headers.append(alias_match.group(1))
            else:
                # Lấy phần cuối cùng sau dấu chấm (table.column -> column)
                col_name = ph.split('.')[-1].strip()
                # Loại bỏ các hàm nếu có (ví dụ COUNT(*))
                if '(' not in col_name and ')' not in col_name:
                    headers.append(col_name)
                elif 'COUNT' in ph.upper() and '*' in ph:
                    headers.append("Số lượng")
                elif 'AVG' in ph.upper():
                    headers.append("Giá trị trung bình")
                elif 'SUM' in ph.upper():
                    headers.append("Tổng giá trị")

    # Nếu không lấy được headers từ query, tạo headers mặc định
    if not headers and data and isinstance(data[0], (list, tuple)):
        num_cols = len(data[0])
        headers = [f"Cột {i+1}" for i in range(num_cols)]
    elif not headers and data and not isinstance(data[0], (list, tuple)): # Trường hợp data là list của các giá trị đơn
        headers = ["Kết quả"]
        data = [[item] for item in data] # Chuyển thành list of lists

    # Kiểm tra lại nếu data[0] không phải là list/tuple (vd: kết quả COUNT(*))
    if data and not isinstance(data[0], (list, tuple)):
        if "COUNT" in query_text_original.upper() and len(headers) == 1: # Dành cho COUNT(*)
             return f"{headers[0]}: {data[0]}"
        data = [[item] for item in data] # Chuyển đổi để tabulate hoạt động


    # Xử lý trường hợp data là list rỗng sau các bước trên
    if not data:
        return "Không có dữ liệu."

    # Định dạng tiền tệ cho các cột có tên 'price', 'gia', 'total', 'tongtien'
    # Và đảm bảo các cột này có dữ liệu số trước khi định dạng
    formatted_data = []
    for row in data:
        new_row = []
        if not isinstance(row, (list, tuple)): # Nếu row không phải list/tuple (vd: [5])
            row_to_process = [row]
        else:
            row_to_process = row

        for i, cell in enumerate(row_to_process):
            col_name_lower = headers[i].lower() if i < len(headers) else ""
            if any(price_keyword in col_name_lower for price_keyword in ['price', 'giá', 'total', 'tổng', 'avg']) :
                try:
                    # Cố gắng chuyển đổi sang float, sau đó định dạng
                    # Nếu là None thì giữ nguyên
                    if cell is None:
                        new_row.append("N/A")
                    else:
                        numeric_cell = float(cell)
                        new_row.append(f"{numeric_cell:,.0f} VNĐ")
                except (ValueError, TypeError):
                    new_row.append(str(cell)) # Nếu không phải số, giữ nguyên
            else:
                new_row.append(str(cell))
        formatted_data.append(new_row)
    try:
        return tabulate(formatted_data, headers=headers, tablefmt="grid")
    except Exception as e:
        print(f"Error tabulating data: {e}. Raw data: {formatted_data}")
        # Fallback nếu tabulate lỗi
        output_str = ""
        if headers:
            output_str += " | ".join(headers) + "\\n"
            output_str += "---".join(["-"*len(h) for h in headers]) + "\\n"
        for row_items in formatted_data:
            output_str += " | ".join(map(str, row_items)) + "\\n"
        return output_str if output_str else "Không thể định dạng kết quả."

In [ ]:
# Hàm xử lý truy vấn của người dùng
def get_answer(user_query_input, chat_history_list):
    processed_user_query = preprocess_query(user_query_input) # xử lý (chuẩn hóa) câu hỏi của người dùng
    identified_query_type = analyze_query_type(processed_user_query) # xác định loại câu hỏi (ví dụ: hỏi giá cao nhất, thấp hơn bao nhiêu, trong khoảng,...).
    print(f"User Query: {user_query_input}")
    print(f"Processed Query: {processed_user_query}")
    print(f"Identified Query Type: {identified_query_type}")

    top_k_results = 15 # Số lượng kết quả mặc định
    db_dialect = db.dialect

    # Xây dựng context từ lịch sử chat
    history_context = ""
    if chat_history_list:
        history_context = "\nLịch sử hội thoại trước đó (để tham khảo, không bắt buộc phải dùng lại toàn bộ):\n"
        for entry in chat_history_list[-3:]: # Lấy 3 lượt thoại gần nhất
             history_context += f"{entry['role']}: {entry['content']}\n"


    # **PROMPT cho AGENT (LLM)**
    system_prefix = f"""Bạn là một Agent AI chuyên gia SQL, nhiệm vụ của bạn là chuyển đổi câu hỏi bằng tiếng Việt của người dùng thành một câu truy vấn SQL ({db_dialect}) chính xác và hiệu quả để truy vấn cơ sở dữ liệu bán hàng.
Sau đó, thực thi truy vấn và trả về kết quả đã được định dạng cho người dùng.

QUAN TRỌNG:
1.  **Hiểu Schema:** Cơ sở dữ liệu có các bảng sau: {', '.join(db.get_usable_table_names())}.
    Schema chi tiết:
    {db_schema}
    Hãy chỉ truy vấn các cột thực sự cần thiết để trả lời câu hỏi. Tránh `SELECT *`.
2.  **Xử Lý Giá Tiền:**
    * Giá sản phẩm trong cột `price` được lưu trữ dưới dạng số nguyên (VND).
    * Khi người dùng nói "300 nghìn", "300k", bạn phải chuyển thành `300000` trong SQL.
    * Khi người dùng nói "1 triệu", "1tr", bạn phải chuyển thành `1000000` trong SQL.
    * Luôn đảm bảo các giá trị so sánh trong SQL là số, không phải chuỗi kèm theo "nghìn" hay "triệu".
3.  **Sử Dụng Tool Tìm Kiếm (`search_products_categories_attributes`):**
    * Nếu câu hỏi của người dùng chứa tên sản phẩm, tên danh mục hoặc các thuộc tính cụ thể mà bạn không chắc chắn về cách viết chính xác trong DB, **BẮT BUỘC** phải sử dụng tool `search_products_categories_attributes` **TRƯỚC TIÊN** để lấy tên/thuật ngữ chính xác.
    * Sau đó, sử dụng tên/thuật ngữ chính xác đó trong mệnh đề `WHERE` của câu truy vấn SQL. Điều này cực kỳ quan trọng để đảm bảo truy vấn đúng.
    * Ví dụ: User: "laptop dell giá bao nhiêu", Tool Input: "laptop dell", Tool Output: "Laptop Dell XPS 13", SQL: `SELECT price FROM products WHERE name = 'Laptop Dell XPS 13'`
4.  **Sinh Truy Vấn SQL:**
    * Chỉ sinh một câu truy vấn SQL duy nhất, hợp lệ về mặt cú pháp cho {db_dialect}.
    * Luôn giới hạn số lượng kết quả trả về (ví dụ: `LIMIT {top_k_results}`) trừ khi người dùng yêu cầu rõ ràng số lượng khác hoặc truy vấn là dạng tổng hợp (COUNT, SUM, AVG không cần LIMIT trừ khi có GROUP BY và muốn giới hạn số nhóm).
    * Sắp xếp kết quả (ORDER BY) một cách hợp lý để hiển thị thông tin hữu ích nhất (ví dụ: theo giá, theo tên, theo ngày mới nhất).
    * Tuyệt đối KHÔNG thực hiện các lệnh DML (INSERT, UPDATE, DELETE, DROP). Nếu người dùng yêu cầu, hãy từ chối.
5.  **Trả Lời:**
    * Sau khi có kết quả từ SQL, hãy trình bày nó một cách rõ ràng, dễ đọc cho người dùng. Nếu là bảng, hãy định dạng bảng.
    * Nếu truy vấn không trả về kết quả, hãy thông báo "Không tìm thấy sản phẩm/thông tin nào phù hợp."
    * Nếu câu hỏi không liên quan đến cơ sở dữ liệu hoặc không thể trả lời, hãy nói "Tôi không thể trả lời câu hỏi này." hoặc "Câu hỏi này nằm ngoài phạm vi kiến thức của tôi."

Loại truy vấn được xác định sơ bộ: {identified_query_type} (dùng để tham khảo, bạn cần phân tích kỹ câu hỏi).
{history_context}
Chỉ trả lời dựa trên thông tin từ cơ sở dữ liệu.
Hãy cẩn thận với các điều kiện AND/OR, LIKE (sử dụng `%` cho tìm kiếm模糊), và JOIN bảng khi cần thiết.
Đảm bảo rằng bạn hiểu đúng ý người dùng trước khi tạo truy vấn.
"""

    try:
        # Chọn ví dụ few-shot dựa trên sự tương đồng ngữ nghĩa
        # Sử dụng lại các examples đã định nghĩa nếu selector gặp vấn đề hoặc để đơn giản hóa
        few_shot_prompt = FewShotPromptTemplate(
            examples=examples,
            example_prompt=PromptTemplate.from_template("Câu hỏi người dùng: {input}\nSQL Query: {query}"),
            prefix=system_prefix,
            suffix="Câu hỏi người dùng: {input}\nSQL Query:", # LLM sẽ hoàn thành phần SQL Query
            input_variables=["input", "agent_scratchpad"], # Thêm agent_scratchpad
        )
    except Exception as e:
        print(f"Lỗi khi tạo FewShotPromptTemplate hoặc chọn ví dụ: {e}")
        # Fallback sử dụng system_prefix trực tiếp nếu few-shot có vấn đề
        final_prompt_template = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(system_prefix + "\nCâu hỏi người dùng: {input}\nSQL Query:"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])


    # Tạo agent
    try:
        from langchain.agents import AgentType

        sql_agent = create_sql_agent(
            llm=llm,
            db=db,
            tools=tools,
            agent_type=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            max_iterations=7,
            handle_parsing_errors="Hãy thử sửa lỗi và chạy lại. Nếu vẫn lỗi, hãy trả lời bạn không thể xử lý.",
            return_intermediate_steps=True,
        )

        print("Invoking SQL Agent...")
        agent_input = {"input": processed_user_query}
        if chat_history_list:
            agent_input["chat_history"] = chat_history_list

        result = sql_agent.invoke(agent_input)
        raw_output = result.get('output', "Không có phản hồi từ agent.")
        print(f"Raw output from agent: {raw_output}")

        # Trích xuất SQL từ intermediate_steps
        generated_sql = ""
        final_answer_from_agent = raw_output # Mặc định là output cuối cùng

        if 'intermediate_steps' in result and result['intermediate_steps']:
            print(f"Intermediate steps: {result['intermediate_steps']}")
            # Duyệt qua các bước để tìm SQL query và kết quả thực thi
            for step in result['intermediate_steps']:
                action, observation = step # action là những gì agent quyết định làm, observation là kết quả
                if hasattr(action, 'tool_input') and isinstance(action.tool_input, str) and "SELECT" in action.tool_input.upper():
                    generated_sql = action.tool_input
                    print(f"SQL generated by agent: {generated_sql}")
                    # Observation thường là kết quả của SQL query
                    final_answer_from_agent = format_sql_result(observation, generated_sql)
                    break # Lấy SQL và kết quả từ bước cuối cùng có SQL
                elif hasattr(action, 'tool') and action.tool == 'sql_db_query':
                     generated_sql = str(action.tool_input) # tool_input có thể là dict hoặc string
                     if isinstance(action.tool_input, dict) and 'query' in action.tool_input:
                         generated_sql = action.tool_input['query']
                     print(f"SQL generated by agent (sql_db_query): {generated_sql}")
                     final_answer_from_agent = format_sql_result(observation, generated_sql)
                     break


        # Nếu không trích xuất được từ intermediate_steps, thử phân tích raw_output
        if not generated_sql and "SELECT" in raw_output.upper() and "FROM" in raw_output.upper():
            select_index = raw_output.upper().find("SELECT")
            end_match = re.search(r";|```", raw_output[select_index:]) # Tìm dấu ; hoặc ```
            if end_match:
                generated_sql = raw_output[select_index : select_index + end_match.start()]
            else:
                generated_sql = raw_output[select_index:]
            print(f"SQL suspected from raw_output: {generated_sql}")
            # Giả định raw_output là câu trả lời cuối cùng nếu không có SQL rõ ràng để tự chạy
            # hoặc nếu agent đã tự chạy và trả về kết quả.
            # Nếu raw_output chứa kết quả dạng bảng mà agent không tự format, chúng ta format nó.
            if "(" in raw_output and ")" in raw_output and "," in raw_output: # Heuristic cho dữ liệu bảng
                 try:
                     potential_data = ast.literal_eval(raw_output)
                     if isinstance(potential_data, list):
                         final_answer_from_agent = format_sql_result(potential_data, generated_sql)
                 except:
                     pass # Giữ nguyên raw_output

        return final_answer_from_agent

    except Exception as e:
        print(f"Lỗi khi thực thi agent hoặc xử lý kết quả: {e}")
        print("Thử thực hiện fallback logic...")
        # **FALLBACK LOGIC MỞ RỘNG**
        try:
            sql_query = ""
            if identified_query_type == "price_less_than":
                price = extract_price_value(processed_user_query)
                if price:
                    sql_query = f"SELECT name, price FROM products WHERE price < {price} ORDER BY price DESC LIMIT {top_k_results};"
            elif identified_query_type == "price_greater_than":
                price = extract_price_value(processed_user_query)
                if price:
                    sql_query = f"SELECT name, price FROM products WHERE price > {price} ORDER BY price ASC LIMIT {top_k_results};"
            elif identified_query_type == "price_range":
                # Cần logic parse 2 giá trị từ processed_user_query
                numbers = re.findall(r'\d+', processed_user_query.replace(',', '').replace('.', ''))
                prices = sorted([int(n) for n in numbers])
                min_price_val, max_price_val = None, None

                if "triệu" in processed_user_query and "nghìn" in processed_user_query: # vd: 1 triệu 500 nghìn đến 2 triệu
                    raw_prices = []
                    parts = re.split(r'\s+đến\s+', processed_user_query)
                    if len(parts) == 2:
                        min_price_val = extract_price_value(parts[0])
                        max_price_val = extract_price_value(parts[1])

                elif len(prices) >= 2:
                    min_p_text = processed_user_query.split("đến")[0]
                    max_p_text = processed_user_query.split("đến")[1] if "đến" in processed_user_query else ""

                    min_price_val = extract_price_value(min_p_text)
                    max_price_val = extract_price_value(max_p_text) if max_p_text else None

                if min_price_val and max_price_val:
                    sql_query = f"SELECT name, price FROM products WHERE price >= {min_price_val} AND price <= {max_price_val} ORDER BY price LIMIT {top_k_results};"
                elif min_price_val:
                     sql_query = f"SELECT name, price FROM products WHERE price >= {min_price_val} ORDER BY price ASC LIMIT {top_k_results};"


            elif identified_query_type == "price_highest":
                limit_match = re.search(r'(\d+)\s*(sản phẩm|sp)', processed_user_query)
                limit = int(limit_match.group(1)) if limit_match else 5
                sql_query = f"SELECT name, price FROM products ORDER BY price DESC LIMIT {limit};"
            elif identified_query_type == "price_lowest":
                limit_match = re.search(r'(\d+)\s*(sản phẩm|sp)', processed_user_query)
                limit = int(limit_match.group(1)) if limit_match else 5
                sql_query = f"SELECT name, price FROM products ORDER BY price ASC LIMIT {limit};"

            elif identified_query_type == "products_in_category":
                # Cố gắng tìm tên danh mục từ retriever nếu có thể
                docs = retriever.get_relevant_documents(processed_user_query)
                category_name_fallback = docs[0].page_content if docs and docs[0].page_content in categories else None
                if category_name_fallback: # Nếu tìm thấy tên danh mục khớp
                    sql_query = f"SELECT p.name, p.price FROM products p JOIN categories c ON p.category_id = c.id WHERE c.name = '{category_name_fallback}' LIMIT {top_k_results};"
                else: # Nếu không, thử tìm với LIKE
                    # Trích xuất phần tên danh mục từ câu hỏi người dùng một cách đơn giản
                    match_cat = re.search(r"(?:thuộc|trong)\s+danh mục\s+['\"]?([^'\"]+)['\"]?", processed_user_query, re.IGNORECASE)
                    if match_cat:
                        cat_name_from_query = match_cat.group(1)
                        sql_query = f"SELECT p.name, p.price FROM products p JOIN categories c ON p.category_id = c.id WHERE c.name LIKE '%{cat_name_from_query}%' LIMIT {top_k_results};"


            elif identified_query_type == "count_products_in_category":
                # Tương tự, trích xuất tên danh mục
                match_cat = re.search(r"danh mục\s+['\"]?([^'\"]+)['\"]?", processed_user_query, re.IGNORECASE)
                if match_cat:
                    cat_name_from_query = match_cat.group(1)
                    # Tìm tên chính xác từ retriever
                    docs = retriever.get_relevant_documents(cat_name_from_query)
                    category_name_exact = next((doc.page_content for doc in docs if doc.page_content in categories), cat_name_from_query)
                    sql_query = f"SELECT COUNT(p.id) FROM products p JOIN categories c ON p.category_id = c.id WHERE c.name = '{category_name_exact}';"


            elif identified_query_type == "product_list_all" or identified_query_type == "product_list":
                 sql_query = f"SELECT name, price, stock_quantity FROM products LIMIT {top_k_results};"

            elif identified_query_type == "list_categories":
                sql_query = f"SELECT name FROM categories LIMIT {top_k_results};"

            elif identified_query_type == "product_by_exact_name":
                match_name = re.search(r"(?:tên là|sản phẩm)\s+['\"]?([^'\"]+)['\"]?", processed_user_query, re.IGNORECASE)
                if match_name:
                    prod_name = match_name.group(1)
                    sql_query = f"SELECT name, price, stock_quantity, description FROM products WHERE name = '{prod_name}';"


            elif identified_query_type == "price_lowest_in_category":
                match_cat = re.search(r"danh mục\s+['\"]?([^'\"]+)['\"]?", processed_user_query, re.IGNORECASE)
                if match_cat:
                    cat_name_from_query = match_cat.group(1).strip()
                    # Sử dụng retriever để lấy tên danh mục chính xác
                    docs = retriever.get_relevant_documents(cat_name_from_query)
                    category_name_exact = next((doc.page_content for doc in docs if doc.page_content.lower() == cat_name_from_query.lower() and doc.page_content in categories), cat_name_from_query)

                    limit_match = re.search(r'(\d+)\s*(sản phẩm|sp)', processed_user_query)
                    limit = int(limit_match.group(1)) if limit_match else 1 # Mặc định là 1 cho "rẻ nhất"

                    if not category_name_exact or category_name_exact not in categories:
                        sql_query = f"SELECT p.name, p.price FROM products p JOIN categories c ON p.category_id = c.id WHERE c.name LIKE '%{cat_name_from_query}%' ORDER BY p.price ASC LIMIT {limit};"
                    else:
                        sql_query = f"SELECT p.name, p.price FROM products p JOIN categories c ON p.category_id = c.id WHERE c.name = '{category_name_exact}' ORDER BY p.price ASC LIMIT {limit};"
            elif identified_query_type == "stock_query_for_product":
                # Trích xuất tên sản phẩm
                prod_name_match = re.search(r"(?:của|cho|là|tên)\s*['\"]([^'\"]+)['\"]", processed_user_query)
                if not prod_name_match: # Thử một pattern khác nếu không có dấu nháy
                    prod_name_match = re.search(r"số lượng\s+(?:của\s+)?(.+?)\s*(?:còn trong kho|trong kho|$)", processed_user_query)

                if prod_name_match:
                    prod_name = prod_name_match.group(1).strip()
                    # Sử dụng retriever để lấy tên sản phẩm chính xác
                    docs = retriever.get_relevant_documents(prod_name)
                    product_name_exact = next((doc.page_content for doc in docs if doc.page_content.lower() == prod_name.lower() and doc.page_content in products), prod_name)

                    if not product_name_exact or product_name_exact not in products:
                        sql_query = f"SELECT name, stock_quantity FROM products WHERE name LIKE '%{prod_name}%' LIMIT 1;" # Giới hạn 1 nếu tìm LIKE
                    else:
                        sql_query = f"SELECT stock_quantity FROM products WHERE name = '{product_name_exact}';"

            if sql_query:
                print(f"Executing Fallback SQL: {sql_query}")
                raw_result = query_as_list(db, sql_query)
                # LOẠI BỎ TRÙNG LẶP KẾT QUẢ TỪ FALLBACK
                if isinstance(raw_result, list) and raw_result:
                    try:
                        # Đảm bảo mỗi phần tử trong raw_result là tuple để có thể cho vào set
                        tuple_results = [tuple(row) if isinstance(row, list) else row for row in raw_result]
                        unique_results = list(dict.fromkeys(tuple_results)) # Giữ thứ tự và loại bỏ trùng lặp
                        # unique_results = list(set(tuple_results)) # Không giữ thứ tự
                        print(f"Fallback result after deduplication: {unique_results}")
                        return format_sql_result(unique_results, sql_query)
                    except TypeError as te:
                         print(f"Could not deduplicate results due to TypeError: {te}. Returning raw.")
                         return format_sql_result(raw_result, sql_query)

                return format_sql_result(raw_result, sql_query)
            else:
                return "Xin lỗi, tôi không thể xử lý yêu cầu này với logic dự phòng hiện tại. Vui lòng thử lại với câu hỏi khác hoặc kiểm tra lại cấu trúc câu hỏi."

        except Exception as fallback_error:
            print(f"Lỗi trong quá trình fallback: {fallback_error}")
            return "Xin lỗi, tôi gặp lỗi nghiêm trọng khi xử lý yêu cầu của bạn. Vui lòng thử lại sau."

In [ ]:
# Tạo giao diện người dùng Gradio
with gr.Blocks(css=".message {white-space: pre-wrap; font-family: monospace;}", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 Chatbot SQL Thông Minh ")
    gr.Markdown("""
    Chatbot này sử dụng mô hình Mistral 7B và Langchain để truy vấn cơ sở dữ liệu sản phẩm bằng tiếng Việt.
    Bạn có thể hỏi về danh mục, giá cả, tồn kho, mô tả sản phẩm và nhiều thông tin khác.
    **Lưu ý:** Đây là phiên bản Beta, một số câu hỏi phức tạp có thể chưa được xử lý hoàn hảo.

    **Ví dụ câu hỏi:**
    - `Liệt kê 5 sản phẩm đắt nhất.`
    - `Sản phẩm nào giá dưới 300 nghìn?`
    - `Cho tôi xem 5 sản phẩm đắt nhất`
    - `Tìm các sản phẩm có giá dưới 200 nghìn.`
    - `Sản phẩm nào có giá từ 500.000 đến 1 triệu đồng?`
    """)

    chatbot_component = gr.Chatbot(
        label="Hộp thoại",
        bubble_full_width=False,
        height=600,
        show_label=False,
        elem_id="chatbot_conversation"
    )

    with gr.Row():
        txt_message = gr.Textbox(
            show_label=False,
            placeholder="Nhập câu hỏi của bạn ở đây...",
            container=False,
            scale=7
        )
        submit_btn = gr.Button("Gửi", variant="primary", scale=1)

    clear_btn = gr.Button("Xóa Lịch Sử Chat", variant="stop")

    # State để lưu trữ lịch sử hội thoại cho Gradio
    chat_history_state = gr.State([])

    # Kết nối sự kiện
    # Khi submit (enter hoặc click nút), gọi hàm chat
    # Inputs: message từ textbox, history từ state
    # Outputs: textbox được xóa, chatbot component được cập nhật, state được cập nhật
    submit_btn.click(
        fn=chat,
        inputs=[txt_message, chat_history_state],
        outputs=[txt_message, chatbot_component, chat_history_state]
    )
    txt_message.submit(
        fn=chat,
        inputs=[txt_message, chat_history_state],
        outputs=[txt_message, chatbot_component, chat_history_state]
    )

    # Nút xóa lịch sử
    def clear_chat_history_func():
        return None, None, [] # Xóa textbox, chatbot UI, và state
    clear_btn.click(
        fn=clear_chat_history_func,
        inputs=None,
        outputs=[txt_message, chatbot_component, chat_history_state],
        queue=False
    )


In [ ]:
#Hàm xử lý cuộc hội thoại
def chat(user_message, current_chat_history_state_list):

    # Chuyển đổi state của Gradio sang định dạng mà get_answer mong muốn (list of dicts)
    formatted_llm_history = []
    if current_chat_history_state_list:
        for user_msg, bot_msg in current_chat_history_state_list:
            if user_msg:
                formatted_llm_history.append({"role": "user", "content": user_msg})
            if bot_msg:
                formatted_llm_history.append({"role": "assistant", "content": bot_msg})

    print(f"\n--- New Turn ---")
    print(f"User message: {user_message}")
    print(f"LLM history for get_answer: {formatted_llm_history}")

    bot_reply_content = "Xin lỗi, đã có lỗi xảy ra và tôi không thể xử lý yêu cầu của bạn." # Default
    try:
        retrieved_answer = get_answer(user_message, formatted_llm_history)
        if retrieved_answer is not None:
            bot_reply_content = retrieved_answer
        else:
            print("Warning: get_answer returned None.")
    except Exception as e:
        print(f"Critical error in chat function calling get_answer: {e}")
        # bot_reply_content sẽ giữ giá trị default

    # Cập nhật state cho Gradio: thêm lượt thoại mới vào cuối
    updated_gradio_history_state = current_chat_history_state_list + [[user_message, bot_reply_content]]

    # Hàm `chat` phải trả về 3 giá trị tương ứng với `outputs` trong .click()/.submit()
    # 1. Giá trị cho txt_message (thường là chuỗi rỗng để xóa input)
    # 2. Giá trị cho chatbot_component (chính là history đã cập nhật)
    # 3. Giá trị cho chat_history_state (cũng là history đã cập nhật để lưu state)
    return "", updated_gradio_history_state, updated_gradio_history_state

In [ ]:
# Khởi chạy ứng dụng
if __name__ == "__main__":
    demo.launch(share=True)